# Setup

In [1]:
# General imports.
from tqdm.notebook import tqdm

# Specific imports.
import pandas as pd

import torch
from torch import nn
from torch.nn import functional as F
from torch.optim import AdamW, SGD
from torch.utils.data import DataLoader
from transformers import get_scheduler, AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from transformers import BertTokenizerFast, BertForSequenceClassification
from torch.profiler import profile, record_function, ProfilerActivity
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split

/Users/hikaruisayama/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/Users/hikaruisayama/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
def tensor(priority):
    if priority == "Low":
        return 0
    elif priority == 'Medium':
        return 1
    elif priority == "High":
        return 2
    else:
        return 3

In [3]:
data = pd.read_csv('customer_support_tickets.csv')
data = data[['Ticket Description', 'Ticket Priority']]
data['Ticket Priority'] = data['Ticket Priority'].apply(tensor)

# Dataset

In [4]:
class CustomTextDataset(torch.utils.data.Dataset):
    def __init__(self, data, model_name="bert-base-uncased"):
        self.data = data
        self.tokenizer = BertTokenizerFast.from_pretrained(model_name)
        self.label = data['Ticket Priority']

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        res = self.tokenizer(text=self.data.iloc[idx].get('Ticket Description'), 
                             padding='max_length', 
                             max_length=500, 
                             truncation=True, 
                             return_tensors='pt')
        return {
            'input_ids': res["input_ids"].squeeze(), 
            'token_type_ids': res["token_type_ids"].squeeze(), 
            'attention_mask': res["attention_mask"].squeeze(),
            "labels": torch.tensor(self.label[idx])
        }

In [5]:
ds = CustomTextDataset(data)

In [6]:
BATCH_SIZE = 20

train_indices, test_indices = train_test_split(
    range(len(ds)),
    test_size=0.1,
)
train_split = Subset(ds, train_indices)
test_split = Subset(ds, test_indices)

train_batches = DataLoader(
    train_split, 
    batch_size=BATCH_SIZE, 
    shuffle=True,
    num_workers = 0,
    pin_memory = True,
    drop_last = True
)
test_batches = DataLoader(test_split, batch_size=BATCH_SIZE)

# Training

In [7]:
class ClassifierHead(nn.Module):
    def __init__(self, hidden_size, num_classes, seq_length):
        super().__init__()
        self.rnn = nn.RNN(768, hidden_size, batch_first=True)
        self.linear = nn.Linear(hidden_size*seq_length, num_classes)

    def forward(self, x):
        x, _ = self.rnn(x)
        x = x.reshape(x.shape[0], -1)
        x = self.linear(x)
        return x

class BERTWithClassifierHead(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.bert = AutoModel.from_pretrained("bert-base-uncased")
        self.classifier = ClassifierHead(20, num_classes, 500)
        
    def forward(self, x):
        x = self.bert(**x)
        x = x.last_hidden_state
        x = self.classifier(x)
        return x

In [8]:
model = BERTWithClassifierHead(num_classes=16)
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = 16, id2label=id2label, label2id=label2id)

In [9]:
# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# model.to(device)
# batch = next(iter(train_batches))
# X = {k: batch[k] for k in batch.keys() if k not in ["label"]}
# y = batch["label"]
# X = {k: v.to(device) for k, v in X.items()}
# y = y.to(device)
# output = model(**X).logits
# print(output.shape, y.shape)

In [10]:
epochs = 3
lr = 5e-5
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Define Loss
criterion = nn.CrossEntropyLoss()

# Define optimizer.
optimizer = AdamW(model.parameters(), lr=lr)
# optimizer = SGD(model.parameters(), lr=lr, momentum=0.9)

# Define LR Scheduler.
num_training_steps = epochs * len(train_batches)
lr_scheduler = get_scheduler(
    name="cosine", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

model.to(device)

BERTWithClassifierHead(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [11]:
# model.to(device)
# with profile(activities=[ProfilerActivity.CPU], profile_memory=True, record_shapes=True) as prof:
#     with record_function("model_inference"):
#         batch = next(iter(train_loader))
#         X = {k: batch[k] for k in batch.keys() if k not in ["label"]}
#         y = batch["label"]
#         X = {k: v.to(device) for k, v in X.items()}
#         y = y.to(device)
#         model(X)
# print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))
# print(prof.key_averages().table(sort_by="self_cpu_memory_usage", row_limit=10))

In [12]:
scaler = torch.cuda.amp.GradScaler()

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(epochs):
    
    running_loss = 0.0
    for i, batch in enumerate(train_batches, 0):
        
        batch = {k: v.to(device) for k, v in batch.items()}
        # Unpack the dictionary.
        X = {k: batch[k] for k in batch.keys() if k not in "labels"}
        y = batch["labels"]

#         One step.
        with torch.autocast(device_type='cuda', dtype=torch.float16):
            outputs = model(X)
            loss = criterion(outputs, y)
            
#         loss.backward()
#         optimizer.step()

        scaler.scale(loss).backward()

        scaler.step(optimizer)
        lr_scheduler.step()
        optimizer.zero_grad()
        
        scaler.update()
        
        progress_bar.update(1)
    
        running_loss += loss.item()
        if i % 50 == 49:    # print every 45 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 50:.3f}')
            running_loss = 0.0
            
print("Finished Training!")


/Users/hikaruisayama/anaconda3/lib/python3.11/site-packages/torch/cuda/amp/grad_scaler.py:126: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


  0%|          | 0/1143 [00:00<?, ?it/s]

/Users/hikaruisayama/anaconda3/lib/python3.11/site-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
scaler = torch.cuda.amp.GradScaler()

batch = next(iter(train_batches))
    
batch = {k: v.to(device) for k, v in batch.items()}
# Unpack the dictionary.
X = {k: batch[k] for k in batch.keys() if k not in "labels"}
y = batch["labels"]
for epoch in range(100):

#         One step.
    with torch.autocast(device_type='cuda', dtype=torch.float16):
        outputs = model(X)
        loss = criterion(outputs, y)
            
#         loss.backward()
#         optimizer.step()

    scaler.scale(loss).backward()

    scaler.step(optimizer)
    lr_scheduler.step()
    optimizer.zero_grad()

    scaler.update()

    progress_bar.update(1)

    print(f'{epoch}=', loss)
            
print("Finished Training!")

In [ ]:
torch.save(model.state_dict(), '/kaggle/working/mbti.pth')

In [ ]:
model = BERTWithClassifierHead(num_classes=16)
# model = AutoModelForSequenceClassification.from_pretrained('bert-base-cased', num_labels=16)
model.load_state_dict(torch.load('/kaggle/working/mbti.pth'))

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
res = tokenizer(text="this is some text hello hello hello", 
                             padding='max_length', 
                             max_length=500, 
                             truncation=True, 
                             return_tensors='pt')


In [ ]:
output = model(res)
print(output)

In [ ]:
m = nn.Softmax(dim=1)
scaled = m(output)
print(scaled)
prediction = torch.argmax(scaled)
prediction

In [ ]:
type(labels[prediction])

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for batch in test_batches:
        X = {k: batch[k] for k in batch.keys() if k not in "label"}
        y = batch["label"]

        X = {k: v.to(device) for k, v in X.items()}
        y = y.to(device)

        outputs = model(**X).logits
        _, predicted = torch.max(outputs, 1)
        _, actual = torch.max(y, 1)

        total += actual.size(0)
        correct += (predicted == actual).sum().item()
    
print(f'Accuracy of the model: {100 * correct // total} %')


In [ ]:
import evaluate

metric = evaluate.load("accuracy")
model.eval()
for batch in test_batches:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    actual = torch.argmax(batch['labels'], dim=-1)
    metric.add_batch(predictions=predictions, references=actual)

metric.compute()